	- LLM Node
		- Prompt
			- Role: Expert Claims Consultant
			- describes how each tool can be used
			- Use tools to do research
			- Determine if the claim in valid via the policy
				- if yes write an email to the insurance company including the scenario and why it is a valid claim
				- if not let the client know why and what they can do to strengthen their claim
		- Access To Tools
	- Tools Node
		- RAG on Uploaded Policy
		- Web Search: 
			- Query what makes for a good claim
			- Explain the technical details in terms of what happened (ie. why wind pushed down a tree)
	- Use LangChain and LangGraph to build a simple agent, with an LLM node that does reasoning and termines what tools to call with what query and a tool node which takes the action

## Imports and Environment Variables

In [274]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [275]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

In [277]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Please enter your Cohere API key!")

In [271]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ClaimAssist"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your Langsmith API Key:")

## Create Retriever for Uploaded Policy

In [272]:
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
embedding_dim = 1536

In [273]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

insurance_policy_client = QdrantClient(":memory:")

insurance_policy_client.create_collection(
    collection_name="insurance_policy",
    vectors_config=VectorParams(size=embedding_dim, distance=Distance.COSINE),
)

insurance_policy_vector_store = QdrantVectorStore(
    client=insurance_policy_client,
    collection_name="insurance_policy",
    embedding=embedding_model,
)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: yB5rj9sL****************************eZ6c. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./data/oklahoma_home_insurance_policy.pdf", mode="single")
insurance_policy_doc = loader.load()

print(insurance_policy_doc)

[Document(metadata={'producer': 'Acrobat Distiller 7.0.5 (Windows)', 'creator': 'Adobe PageMaker 7.0', 'creationdate': '2007-10-18T14:41:31-05:00', 'subject': 'HO3', 'author': 'WPD4481', 'keywords': '', 'moddate': '2007-10-18T14:41:35-04:00', 'title': 'HO3', 'source': './data/oklahoma_home_insurance_policy.pdf', 'total_pages': 29}, page_content="HO-4 (01-07)\nFor information regarding this policy, please\ncontact your Shelter Insurance Agent.\nHOMEOWNERS'\nINSURANCE\nPOLICY\nSHELTER INSURANCE COMPANIES\nHome Office: Columbia, MO 65218-0001\ndfg\nHO-4\n(01-07)\nTO OUR CUSTOMERS – PLEASE NOTE\nPlease read this policy carefully.  If you have\nquestions, contact your Shelter Agent for answers.\nNo agent can know your exact coverage needs or\nbudget considerations, so it is your responsibility to\nexamine the policy and make sure it provides the\ntypes of coverage you need in the amounts you\nrequested.\nIf you suffer a loss, please read this policy again so\nthat you will be reminded of yo

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=1000,
                    chunk_overlap=200
                )

insurance_policy_chunks = text_splitter.split_documents(insurance_policy_doc)
len(insurance_policy_chunks)

116

In [ ]:
_ = insurance_policy_vector_store.add_documents(documents=insurance_policy_chunks)

In [ ]:
insurance_policy_retriever = insurance_policy_vector_store.as_retriever(search_kwargs={"k": 5})

In [ ]:
insurance_policy_retriever.invoke("What is the coverage for a fire?")

[Document(metadata={'producer': 'Acrobat Distiller 7.0.5 (Windows)', 'creator': 'Adobe PageMaker 7.0', 'creationdate': '2007-10-18T14:41:31-05:00', 'subject': 'HO3', 'author': 'WPD4481', 'keywords': '', 'moddate': '2007-10-18T14:41:35-04:00', 'title': 'HO3', 'source': './data/oklahoma_home_insurance_policy.pdf', 'total_pages': 29, '_id': 'c3ce4571486e48998ccda634748c488a', '_collection_name': 'insurance_policy'}, page_content='No coverage is provided  under this section for\nthe removal of trees, shrubs, plants, and lawns.\nCoverage for those items, if any, is provided\nunder a specific heading elsewhere in this policy.\nNo coverage is provided under this section for\nthe fees or expenses you incur for tearing off,\nor tearing out, the remains of covered property.\nCoverage for those items, if any, is a part of the\nrestoration cost associated with the specific loss.\n2. Fire Department Charges\nWe will pay up to $500 for your liability under\nan agreement for service charges made by a

## BUILD RAG TOOLS

In [ ]:
from typing import Annotated
from langchain_core.tools import tool

@tool
def retrieve_insurance_policy(
    query: Annotated[str, "query to ask the retrieve insurance policy tool"]
    ):
  """Use Retrieval Augmented Generation to retrieve information insurance policy clauses to determine if a claim is covered"""
  return insurance_policy_retriever.invoke(query)

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=5)

/var/folders/r5/wsbp5_jj7lsfh6rxrw7p7lv00000gn/T/ipykernel_16029/1911882425.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(max_results=5)


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
tool_belt = [retrieve_insurance_policy, tavily_tool]
llm_with_tools = llm.bind_tools(tool_belt)

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
  user_input: str
  messages: Annotated[list, add_messages]

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
system_prompt = """
You are a highly experienced Insurance Claim Consultant.

Your job is to evaluate whether a user's insurance claim is valid, based on the uploaded insurance policy. You have access to two tools:

1. **RAG Search on Insurance Policy** – Use this to search and retrieve relevant clauses from the user’s uploaded policy.
2. **Web Search Tool** – Use this to research any external facts (e.g., explanations of storm mechanics, typical standards for valid claims, or definitions of insurance terminology) to help clarify or strengthen your response.

---

📌 **Tool Usage Guidelines**

- **Always use the RAG Search first.** Your primary responsibility is to validate the claim strictly based on the user’s uploaded policy. Use this tool as often as needed to understand the policy and how it applies to the user's claim.
- **Only use the Web Search if needed**, after the RAG Search:
  - If the policy is unclear or lacks specific wording on the situation described
  - If the user mentions a complex or technical situation that may require external explanation (e.g., “derecho wind event”, “burst pipe due to negative pressure”)
- **Do not overuse the Web Search.** Your main objective is to help the user understand whether their claim is supported by the actual policy document.

---

🧠 **Agent Behavior**

1. **Parse the user’s claim** — Identify the scenario, cause of loss, location, date, and keywords (e.g., water damage, wind, theft).
2. **Query the Insurance Policy using the RAG Search tool** to find clauses that mention covered perils, exclusions, and conditions.
3. **Determine if the claim is valid or not.** Base your decision primarily on what the policy says.
4. If necessary, **use the Web Search tool** to clarify any uncertain facts or explain why a technical detail supports or weakens the claim.
5. **Generate a response to the user:**
   - If the claim is valid: write a professional email the user can send to their insurance company, referencing relevant clauses.
   - If the claim is not valid: explain why not, and give clear, actionable suggestions on what the user can do to strengthen or reframe the claim.

---

🔒 Always ground your decision in the uploaded policy first, and be concise, helpful, and accurate.
"""

def prepare_input(state):
  messages = []
  messages.append(SystemMessage(content=system_prompt))
  messages.append(HumanMessage(content=state["user_input"]))
  return {"messages": messages}


In [ ]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = llm_with_tools.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

## Build The Graph

In [ ]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("prepare_input", prepare_input)
uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)
uncompiled_graph.set_entry_point("prepare_input")

def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

uncompiled_graph.add_edge("prepare_input", "agent")
uncompiled_graph.add_edge("action", "agent")


claim_consultant_agent = uncompiled_graph.compile()


## Testing Initial Solution

In [ ]:
query = """
A sudden freeze on January 17th caused a pipe in our upstairs bathroom to burst overnight. We woke up to water leaking through the ceiling into the kitchen. We immediately shut off the water and contacted a plumber. Since the damage was accidental and not due to neglect, we are submitting a claim for the cost of plumbing repair and water mitigation services.
"""

input = {"user_input": query}
async for chunk in claim_consultant_agent.astream(input, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'prepare_input'
[SystemMessage(content="\nYou are a highly experienced Insurance Claim Consultant.\n\nYour job is to evaluate whether a user's insurance claim is valid, based on the uploaded insurance policy. You have access to two tools:\n\n1. **RAG Search on Insurance Policy** – Use this to search and retrieve relevant clauses from the user’s uploaded policy.\n2. **Web Search Tool** – Use this to research any external facts (e.g., explanations of storm mechanics, typical standards for valid claims, or definitions of insurance terminology) to help clarify or strengthen your response.\n\n---\n\n📌 **Tool Usage Guidelines**\n\n- **Always use the RAG Search first.** Your primary responsibility is to validate the claim strictly based on the user’s uploaded policy. Use this tool as often as needed to understand the policy and how it applies to the user's claim.\n- **Only use the Web Search if needed**, after the RAG Search:\n  - If the policy is unclear or lacks 

## Improved Solution - Contextual Compression Retrieval

In [278]:
adjusted_insurance_policy_retriever_retriever = insurance_policy_vector_store.as_retriever(search_kwargs={"k": 20})

In [285]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=adjusted_insurance_policy_retriever_retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

ImportError: cannot import name 'ChatResponse' from 'cohere.types' (/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/cohere/types/__init__.py)

In [ ]:
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever

def retrieve_adjusted(state):
    # FlashRank - works offline, no API key needed
    compressor = FlashrankRerank()

    # Create compression retriever with your base retriever
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=adjusted_insurance_policy_retriever_retriever  # Your existing retriever
    )
    retrieved_docs = compression_retriever.invoke(state["question"])
    return {"context" : retrieved_docs}

In [288]:
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever

# FlashRank - works offline, no API key needed
compressor = FlashrankRerank()

# Create compression retriever with your base retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=adjusted_insurance_policy_retriever_retriever  # Your existing retriever
)

In [289]:
from typing import Annotated
from langchain_core.tools import tool

@tool
def retrieve_insurance_policy_advanced(
    query: Annotated[str, "query to ask the retrieve insurance policy tool"]
    ):
  """Use Retrieval Augmented Generation to retrieve information insurance policy clauses to determine if a claim is covered"""
  return compression_retriever.invoke(query)

In [294]:
llm_advanced = ChatOpenAI(model="gpt-4o-mini")
tool_belt_advanced = [retrieve_insurance_policy_advanced, tavily_tool]
llm_with_tools_advanced = llm_advanced.bind_tools(tool_belt_advanced)
tool_node_advanced = ToolNode(tool_belt_advanced)

In [301]:
from langgraph.prebuilt import ToolNode

def call_model_advanced(state):
  messages = state["messages"]
  response = llm_with_tools_advanced.invoke(messages)
  return {"messages" : [response]}

tool_node_advanced = ToolNode(tool_belt_advanced)

In [302]:
from langgraph.graph import StateGraph, END

uncompiled_advanced_graph = StateGraph(AgentState)

uncompiled_advanced_graph.add_node("prepare_input", prepare_input)
uncompiled_advanced_graph.add_node("agent", call_model_advanced)
uncompiled_advanced_graph.add_node("action", tool_node_advanced)
uncompiled_advanced_graph.set_entry_point("prepare_input")

uncompiled_advanced_graph.add_conditional_edges(
    "agent",
    should_continue
)

uncompiled_advanced_graph.add_edge("prepare_input", "agent")
uncompiled_advanced_graph.add_edge("action", "agent")

advanced_claim_consultant_agent = uncompiled_advanced_graph.compile()

In [303]:
query = """
A sudden freeze on January 17th caused a pipe in our upstairs bathroom to burst overnight. We woke up to water leaking through the ceiling into the kitchen. We immediately shut off the water and contacted a plumber. Since the damage was accidental and not due to neglect, we are submitting a claim for the cost of plumbing repair and water mitigation services.
"""

input = {"user_input": query}
async for chunk in advanced_claim_consultant_agent.astream(input, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'prepare_input'
[SystemMessage(content="\nYou are a highly experienced Insurance Claim Consultant.\n\nYour job is to evaluate whether a user's insurance claim is valid, based on the uploaded insurance policy. You have access to two tools:\n\n1. **RAG Search on Insurance Policy** – Use this to search and retrieve relevant clauses from the user’s uploaded policy.\n2. **Web Search Tool** – Use this to research any external facts (e.g., explanations of storm mechanics, typical standards for valid claims, or definitions of insurance terminology) to help clarify or strengthen your response.\n\n---\n\n📌 **Tool Usage Guidelines**\n\n- **Always use the RAG Search first.** Your primary responsibility is to validate the claim strictly based on the user’s uploaded policy. Use this tool as often as needed to understand the policy and how it applies to the user's claim.\n- **Only use the Web Search if needed**, after the RAG Search:\n  - If the policy is unclear or lacks 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0smHCUdNgrhcvfJDchcMHLlz', 'function': {'arguments': '{"query":"burst pipe, water damage, covered perils, exclusions, loss due to sudden freeze"}', 'name': 'retrieve_insurance_policy_advanced'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 673, 'total_tokens': 707, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C0cWl2im2LzvP6GQirUE59yrAHZnp', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3d143766-b2fe-4fcf-b447-3220773169ca-0', tool_calls=[{'name': 'retrieve_insurance_policy_advanced', 'args': {'query': 'burst pipe, water 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Receiving update from node: 'action'
[ToolMessage(content="[Document(metadata={'id': 3, 'relevance_score': np.float32(0.9994857), 'producer': 'Acrobat Distiller 7.0.5 (Windows)', 'creator': 'Adobe PageMaker 7.0', 'creationdate': '2007-10-18T14:41:31-05:00', 'subject': 'HO3', 'author': 'WPD4481', 'keywords': '', 'moddate': '2007-10-18T14:41:35-04:00', 'title': 'HO3', 'source': './data/oklahoma_home_insurance_policy.pdf', 'total_pages': 29, '_id': '1822883913f1411daedd93b41333647a', '_collection_name': 'insurance_policy'}, page_content='12\\nfrom home if the student has been at\\nthat dwelling at any time during the 45\\ndays immediately before the loss;\\n(2) watercraft and their equipment; and\\n(3) trailers.\\n10. Breakage of Glass\\nWe cover damage to personal property caused\\nby breakage of glass constituting a part of any\\nbuilding on the insured premises, however we\\ndo not cover loss or damage to the glass itself\\nunder this coverage.\\n11. Volcanic Eruption\\nThis peril does

...

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZsemQGUbdPcc2oH1N4a7njDZ', 'function': {'arguments': '{"query":"water damage from burst pipe, coverage on water mitigation, plumbing repairs"}', 'name': 'retrieve_insurance_policy_advanced'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 1990, 'total_tokens': 2020, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C0cWoRkcP9aYzzHFCIlzLfkDifBCx', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--01b760ef-e13b-4c3c-8c31-32273dc53cf8-0', tool_calls=[{'name': 'retrieve_insurance_policy_advanced', 'args': {'query': 'water damage from b

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Receiving update from node: 'action'
[ToolMessage(content="[Document(metadata={'id': 2, 'relevance_score': np.float32(0.9988929), 'producer': 'Acrobat Distiller 7.0.5 (Windows)', 'creator': 'Adobe PageMaker 7.0', 'creationdate': '2007-10-18T14:41:31-05:00', 'subject': 'HO3', 'author': 'WPD4481', 'keywords': '', 'moddate': '2007-10-18T14:41:35-04:00', 'title': 'HO3', 'source': './data/oklahoma_home_insurance_policy.pdf', 'total_pages': 29, '_id': '0cdaf346918a4a4482dc1e9a44608726', '_collection_name': 'insurance_policy'}, page_content='erosion, movement resulting from improper\\nconstruction or compaction, site selection, or\\nany other force. We do cover accidental direct\\nphysical loss that occurs subsequent to any of\\nthese causes if that loss is caused by theft, fire,\\nor explosion.\\n3. (a) Water, or water borne contaminants or\\nmaterials, that flows on, or under, the\\nsurface of the ground; waves; tidal\\nwaters; or overflow of a stream or any\\nbody of water. We do not cover

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Receiving update from node: 'agent'
[AIMessage(content='Based on your description of the incident and the details of your insurance policy, your claim for the damage caused by the burst pipe resulting from the sudden freeze appears to be valid. Here’s how the policy supports your claim:\n\n1. **Covered Peril**: Most standard homeowners policies cover water damage caused by sudden and accidental pipe bursts. Since your pipe burst was due to a sudden freeze and was not a result of neglect, this aligns with typical policy coverage.\n\n2. **Water Damage Coverage**: You mentioned incurring expenses for plumbing repairs and water mitigation services. The policy generally will cover damage that results from a sudden and accidental event like your pipe bursting, as long as these events meet the definitions laid out in the policy.\n\n3. **No Exclusions**: There are clauses that specify when water damage is covered—such as damage stemming from a burst pipe. The policy does not seem to have speci

## GENERATE GOLDEN DATA SET

In [ ]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [ ]:
# Create custom query distribution focusing on claim scenarios
from ragas.testset.synthesizers import MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

# Custom query distribution - prioritize single-hop claim scenarios
claim_query_distribution = [
    (claim_synthesizer, 0.7),  # 70% single-hop claim scenarios
    (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.15),  # 15% multi-hop abstract
    (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.15),  # 15% multi-hop specific
]

# Generate dataset using the custom distribution
from ragas.testset import TestsetGenerator

claim_generator = TestsetGenerator(
    llm=generator_llm, 
    embedding_model=generator_embeddings
)

# Generate claim dataset
claim_dataset = claim_generator.generate_with_langchain_docs(
    insurance_policy_chunks[:10],  # Use more chunks for variety
    testset_size=15,
    query_distribution=claim_query_distribution
)

In [ ]:
from ragas.testset.persona import Persona

# Persona 1: First-Time Homeowner
persona_first_time_homeowner = Persona(
    name="First-Time Homeowner",
    role_description="Recently bought a home and wants to file a claim for the first time. Is unsure about what types of events are covered and unfamiliar with policy exclusions."
)

# Persona 2: Storm Victim
persona_storm_victim = Persona(
    name="Storm Victim",
    role_description="Experienced a severe storm and now wants to file a claim for damages such as a fallen tree or roof leaks."
)

# Persona 3: Diligent Planner
persona_diligent_planner = Persona(
    name="Diligent Planner",
    role_description="Wants to file a claim in a thorough and informed way, carefully reviewing the policy and anticipating potential denial reasons."
)

# Persona 4: Elderly Retiree
persona_elderly_retiree = Persona(
    name="Elderly Retiree",
    role_description="Has lived in the same home for decades and wants to file a claim for what they believe is covered damage, though the issue may have developed over time."
)

# Persona 5: Frustrated Policyholder
persona_frustrated_policyholder = Persona(
    name="Frustrated Policyholder",
    role_description="Has had claims denied in the past and is now filing a new claim, determined to get clear justification and fair treatment from the insurance provider."
)

# Persona 6: Busy Professional
persona_busy_professional = Persona(
    name="Busy Professional",
    role_description="Has limited time to read policy details and wants to quickly file a claim for recent damage, hoping it is covered."
)

personas = [
    persona_first_time_homeowner,
    persona_storm_victim,
    persona_diligent_planner,
    persona_elderly_retiree,
    persona_frustrated_policyholder,
    persona_busy_professional,
]

In [ ]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings, persona_list=personas)
dataset = generator.generate_with_langchain_docs(insurance_policy_chunks[:2], testset_size=10)

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

## Langsmith Data Set

In [ ]:
from langsmith import Client

client = Client()

dataset_name = "Insurance Claim Data 2"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Insurance Claim Data 2"
)

In [ ]:
for data_row in claim_dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

In [ ]:

from ragas.testset.generator.query.single_hop import SingleHopQuery

class ClaimScenarioQuery(SingleHopQuery):
    query_prompt_template = """
You are generating realistic insurance **claim descriptions** based on the given context.

Each claim should:
- Be 3–5 sentences long
- Sound like a real homeowner trying to describe an incident to their insurance company
- Mention a date or timeframe
- Be directly related to the content of the provided context (e.g., perils, exclusions, coverage)
- Avoid asking questions — this is not a Q&A task

Only return a single claim scenario per output. Begin immediately with the claim. Do not include headers like "Claim:" or "Scenario:".

Context:
{context}

Claim:
"""

ModuleNotFoundError: No module named 'ragas.testset.generator'

In [ ]:
# Create custom claim scenario synthesizer using prompt modification
from ragas.testset.synthesizers.single_hop import SingleHopQuerySynthesizer,
from ragas.llms import LangchainLLMWrapper
from langchain_core.prompts import PromptTemplate

class ClaimScenarioQuerySynthesizer(SingleHopQuerySynthesizer):
    def __init__(self, llm):
        super().__init__(llm=llm)
        
        # Custom prompt for generating insurance claim descriptions
        claim_prompt_template = """
You are generating realistic insurance claim descriptions based on the given context.

Each claim should:
- Be a realistic scenario a homeowner would describe when filing an insurance claim
- Be 3-5 sentences long describing an incident or damage
- Include specific details like dates, weather conditions, or circumstances
- Be directly related to the coverage, exclusions, or perils mentioned in the context
- Sound like natural language from a homeowner, not technical insurance language
- Focus on describable incidents that would require policy evaluation

Do not generate questions or queries. Generate actual claim descriptions.

Context: {context}

Generate a realistic insurance claim description:
"""
        
        # Override the default prompt
        self.generate_query_reference_prompt = PromptTemplate(
            input_variables=["context"],
            template=claim_prompt_template
        )

# Initialize the custom synthesizer
claim_synthesizer = ClaimScenarioQuerySynthesizer(llm=generator_llm)

TypeError: Can't instantiate abstract class ClaimScenarioQuerySynthesizer without an implementation for abstract method '_generate_scenarios'

In [305]:
# Generate claim dataset
claim_dataset = claim_generator.generate_with_langchain_docs(
    insurance_policy_chunks[:10],  # Use more chunks for variety
    testset_size=15,
    query_distribution=claim_query_distribution
)

Applying SummaryExtractor:   0%|          | 0/9 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Applying CustomNodeFilter:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/29 [00:00<?, ?it/s]

...

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

INFO:ragas.testset.synthesizers.multi_hop.abstract:found 120 clusters
INFO:ragas.testset.synthesizers.multi_hop.specific:found 3 clusters
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.o

Generating Samples:   0%|          | 0/16 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

In [ ]:
# Generate claim dataset
claim_dataset = claim_generator.generate_with_langchain_docs(
    insurance_policy_chunks[:10],  # Use more chunks for variety
    testset_size=15,
    query_distribution=claim_query_distribution

In [ ]:
import typing as t

from pydantic import BaseModel

from ragas.prompt import PydanticPrompt
from ragas.testset.persona import Persona

from ragas.testset.synthesizers.single_hop.prompts import QueryCondition, GeneratedQueryAnswer


class CustomQueryAnswerGenerationPrompt(PydanticPrompt[QueryCondition, GeneratedQueryAnswer]):
    instruction: str = (
        "Generate a realistic insurance claim description and determine its validity based on the provided policy context. "
        "The claim should sound like a real homeowner describing an incident to their insurance company, and the response "
        "should evaluate if the claim is covered based solely on the policy terms."
        "### Instructions:\n"
        "1. **Generate a Claim**: Based on the context, persona, term, style, and length, create a realistic insurance claim "
        "that describes an incident related to the policy coverage or exclusions. Include specific details like dates and damages.\n"
        "2. **Generate a Response**: Using only the content from the provided context, determine whether the claim would be "
        "covered under the policy. Cite specific policy language to justify the response.\n"
    )
    input_model: t.Type[QueryCondition] = QueryCondition 
    output_model: t.Type[GeneratedQueryAnswer] = GeneratedQueryAnswer
    examples: t.List[t.Tuple[QueryCondition, GeneratedQueryAnswer]] = [
        (
            QueryCondition(
                persona=Persona(
                    name="Homeowner",
                    role_description="A policyholder filing an insurance claim.",
                ),
                term="water damage",
                query_style="Formal",
                query_length="Medium", 
                context="We do not cover loss caused by continuous or repeated seepage or leakage of water or steam from a plumbing system over a period of weeks, months or years. We do cover sudden and accidental discharge or overflow of water or steam from a plumbing system.",
            ),
            GeneratedQueryAnswer(
                query="On March 15th, I discovered significant water damage in my kitchen from a burst pipe under the sink. The pipe suddenly burst while I was at work, flooding the kitchen floor and damaging the cabinets. I shut off the water immediately when I got home and called a plumber for emergency repairs.",
                answer="Based on the policy terms, this claim would be covered as it involves a sudden and accidental discharge of water from a plumbing system. The policy specifically covers 'sudden and accidental discharge or overflow of water or steam from a plumbing system.' This is distinct from gradual water damage that occurs over weeks or months, which would be excluded.",
            ),
        ),
    ]


In [ ]:
from ragas.testset.synthesizers import SingleHopSpecificQuerySynthesizer
synthesizer = SingleHopSpecificQuerySynthesizer(llm=generator_llm)

synthesizer.generate_query_reference_prompt = CustomQueryAnswerGenerationPrompt()

print(query)

QueryAnswerGenerationPrompt(
  "name": "query_answer_generation_prompt",
  "instruction": "\nYou are generating a **realistic insurance claim description**, not a question.\n\nEach claim should:\n- Be 3–5 sentences long\n- Sound like a real homeowner trying to describe an incident to their insurance company\n- Mention a date or timeframe\n- Be directly related to the content of the provided context (e.g., perils, exclusions, coverage)\n- Avoid asking questions — this is not a Q&A task\n\nOnly return a single claim scenario per output. Begin immediately with the claim. Do not include headers like \"Claim:\" or \"Scenario:\".\n",
  "examples": [
    [
      {
        "persona": {
          "name": "Software Engineer",
          "role_description": "Focuses on coding best practices and system design."
        },
        "term": "microservices",
        "query_style": "Formal",
        "query_length": "Medium",
        "context": "Microservices are an architectural style where applications

In [ ]:
query.instruction = """
You are generating a **realistic insurance claim description**, not a question.

Each claim should:
- Be 3–5 sentences long
- Sound like a real homeowner trying to describe an incident to their insurance company
- Mention a date or timeframe
- Be directly related to the content of the provided context (e.g., perils, exclusions, coverage)
- Avoid asking questions — this is not a Q&A task

Only return a single claim scenario per output. Begin immediately with the claim. Do not include headers like "Claim:" or "Scenario:".
"""

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
claim_dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"I recently moved to Columbia and noticed some water stains on my ceiling after a heavy rainstorm on April 10th. Upon inspection, I found that the roof had a leak, which caused water to seep into my attic and damage some of my stored belongings. I had a roofing contractor repair the leak on April 12th. I am requesting coverage for the damages caused by this leak.","[HO-4 (01-07)\nFor information regarding this policy, please\ncontact your Shelter Insurance Agent.\nHOMEOWNERS'\nINSURANCE\nPOLICY\nSHELTER INSURANCE COMPANIES\nHome Office: Columbia, MO 65218-0001\ndfg\nHO-4\n(01-07)\nTO OUR CUSTOMERS – PLEASE NOTE\nPlease read this policy carefully. If you have\nquestions, contact your Shelter Agent for answers.\nNo agent can know your exact coverage needs or\nbudget considerations, so it is your responsibility to\nexamine the policy and make sure it provides the\ntypes of coverage you need in the amounts you\nrequested.\nIf you suffer a loss, please read this policy again so\nthat you will be reminded of your rights and\nobligations. It is very important for you to recognize\nthat this insurance policy is a legally binding\ncontract. If any insured fails to perform an\nobligation required by this policy, the coverage\nwhich it might otherwise provide could be lost.\n HO-4 (01-07)\nTHE INDEX\nWHERE YOU CAN FIND IT]","Based on the policy terms, this claim may not be covered. The policy does not specify coverage for roof leaks or water damage resulting from weather-related causes. Since the damage was caused by a leak during a storm, and the policy emphasizes the importance of reviewing coverage details with your Shelter Agent, it is likely that this type of damage is excluded unless explicitly covered. Therefore, without confirmation of coverage for roof leaks caused by weather, this claim would probably not be eligible for reimbursement under the current policy.",single_hop_specifc_query_synthesizer
1,"On April 10th, I accidentally damaged my roof during a storm, causing a leak that led to water seeping into my attic and damaging some personal belongings. I believe this damage was caused by the storm, which was sudden and unexpected. I am requesting coverage for repairs and replacement of damaged items.","[contract. If any insured fails to perform an\nobligation required by this policy, the coverage\nwhich it might otherwise provide could be lost.\n HO-4 (01-07)\nTHE INDEX\nWHERE YOU CAN FIND IT\nDeclarations - Your Name, Location of Your Residence, Policy Period, Limits of Liability and Deductibles\nBeginning on Page\nAgreement ............................................................................................................................................................. 2\nDefinitions ............................................................................................................................................................. 2\nGeneral Agreements applicable to entire policy .................................................................................................... 8\nSection I - Property Protection ........................................................................................................................... 10]","Based on the policy terms, this claim may not be covered. The policy states that if any insured fails to perform an obligation required by the policy, the coverage could be lost. Since the claim involves damage caused by a storm, which is typically considered an insured peril, coverage could be applicable. However, the policy also emphasizes the importance of adhering to obligations and proper maintenance. If the damage resulted from neglect or failure to perform required obligations, the claim might be denied. Without evidence that all obligations were met, the validity of this claim cannot be confirmed solely based on the provided policy language.",single_hop_specifc_query_synthesizer


In [ ]:
import typing as t

from pydantic import BaseModel

from ragas.prompt import PydanticPrompt
from ragas.testset.persona import Persona

from ragas.testset.synthesizers.single_hop.prompts import QueryCondition, GeneratedQueryAnswer


class CustomQueryAnswerGenerationPrompt(PydanticPrompt[QueryCondition, GeneratedQueryAnswer]):
    instruction: str = (
        "Generate a realistic insurance claim description and determine its validity based on the provided policy context. "
        "The claim should sound like a real homeowner describing an incident to their insurance company, and the response "
        "should evaluate if the claim is covered based solely on the policy terms."
        "### Instructions:\n"
        "1. **Generate a Claim**: Based on the context, persona, term, style, and length, create a realistic insurance claim "
        "that describes an incident related to the policy coverage or exclusions. Include specific details like dates and damages. Some claims should be valid, some should be invalid.\n"
        "2. **Generate a Response**: Using only the content from the provided context, determine whether the claim would be "
        "covered under the policy. Cite specific policy language to justify the response. If not covered, explain why.\n"
        "At the beginning of the response, state whether the claim is valid or invalid. Afterwards give an explanation why. If it is invalid, give suggestions for how to make it valid.\n"
    )
    input_model: t.Type[QueryCondition] = QueryCondition 
    output_model: t.Type[GeneratedQueryAnswer] = GeneratedQueryAnswer
    examples: t.List[t.Tuple[QueryCondition, GeneratedQueryAnswer]] = [
        (
            QueryCondition(
                persona=Persona(
                    name="Homeowner",
                    role_description="A policyholder filing an insurance claim.",
                ),
                term="water damage",
                query_style="Formal",
                query_length="Medium", 
                context="We do not cover loss caused by continuous or repeated seepage or leakage of water or steam from a plumbing system over a period of weeks, months or years. We do cover sudden and accidental discharge or overflow of water or steam from a plumbing system.",
            ),
            GeneratedQueryAnswer(
                query="On March 15th, I discovered significant water damage in my kitchen from a burst pipe under the sink. The pipe suddenly burst while I was at work, flooding the kitchen floor and damaging the cabinets. I shut off the water immediately when I got home and called a plumber for emergency repairs.",
                answer="Based on the policy terms, this claim would be covered as it involves a sudden and accidental discharge of water from a plumbing system. The policy specifically covers 'sudden and accidental discharge or overflow of water or steam from a plumbing system.' This is distinct from gradual water damage that occurs over weeks or months, which would be excluded.",
            ),
        ),
    ]

custom_single_hop_synthesizer = SingleHopSpecificQuerySynthesizer(llm=generator_llm)

custom_single_hop_synthesizer.generate_query_reference_prompt = CustomQueryAnswerGenerationPrompt()


In [ ]:
from ragas.testset.synthesizers.multi_hop.prompts import QueryConditions, GeneratedQueryAnswer

class MultiHopCustomQueryAnswerGenerationPrompt(
    PydanticPrompt[QueryConditions, GeneratedQueryAnswer]
):
    instruction: str = (
        "Generate a complex insurance claim and determine its validity based on multiple sections of the insurance policy. "
        "The claim should require connecting information from different parts of the policy to determine coverage. "
        "The themes represent key policy concepts that should be incorporated into the claim scenario."
        "### Instructions:\n"
        "1. **Generate a Multi-Part Claim**: Create a detailed insurance claim that involves multiple aspects of coverage "
        "from different policy sections (marked as `<1-hop>`, `<2-hop>`, etc.). The claim should realistically describe "
        "an incident that requires analyzing multiple policy provisions. Some claims should be valid, some should be invalid.\n"
        "2. **Generate a Coverage Analysis**: Using only the provided policy sections, determine whether the claim would be "
        "covered. Reference specific language from multiple sections to justify the conclusion. At the beggining of the response, state whether the claim is valid or invalid. After words give an explanation why. If it is invalid, give suggestions for how to make it valid.\n"
        "3. **Multi-Hop Context Tags**:\n"
        "   - Each policy section is tagged as `<1-hop>`, `<2-hop>`, etc.\n"
        "   - The claim and analysis must meaningfully connect provisions from multiple sections."
    )
    input_model: t.Type[QueryConditions] = QueryConditions
    output_model: t.Type[GeneratedQueryAnswer] = GeneratedQueryAnswer
    examples: t.List[t.Tuple[QueryConditions, GeneratedQueryAnswer]] = [
        (
            QueryConditions(
                persona=Persona(
                    name="Homeowner",
                    role_description="A policyholder filing a complex insurance claim.",
                ),
                themes=["Water Damage", "Vehicle Coverage"],
                query_style="Formal",
                query_length="Medium",
                context=[
                    "<1-hop> We do not cover loss caused by continuous or repeated seepage of water over weeks or months. We do cover sudden and accidental discharge of water.",
                    "<2-hop> We do not cover vehicles that are or were motorized, except those used primarily to service the residence premises.",
                ],
            ),
            GeneratedQueryAnswer(
                query="Last week during a severe storm, my riding lawnmower was damaged when my basement flooded suddenly due to a backed-up drain. The water rose quickly and submerged the mower before I could move it. The mower was stored in the basement because I use it weekly to maintain my property. Both the flooding and vehicle damage occurred within a few hours.",
                answer=(
                    "This claim requires analyzing both water damage and vehicle coverage provisions. The water damage would be covered since it was a 'sudden and accidental discharge' rather than gradual seepage. Additionally, while motorized vehicles are generally excluded, the riding lawnmower would be covered because it falls under the exception for vehicles 'used primarily to service the residence premises.' Therefore, based on both policy sections, this claim would likely be approved for both the water damage and mower damage."
                ),
            ),
        ),
    ]

custom_multi_hop_abstract_synthesizer = MultiHopAbstractQuerySynthesizer(llm=generator_llm)
custom_multi_hop_abstract_synthesizer.generate_query_reference_prompt = MultiHopCustomQueryAnswerGenerationPrompt()

custom_multi_hop_specific_synthesizer = MultiHopSpecificQuerySynthesizer(llm=generator_llm)
custom_multi_hop_specific_synthesizer.generate_query_reference_prompt = MultiHopCustomQueryAnswerGenerationPrompt()

In [347]:
claim_query_distribution = [
    (custom_single_hop_synthesizer, 0.25),
    (custom_multi_hop_abstract_synthesizer, 0.375),
    (custom_multi_hop_specific_synthesizer, 0.375),
]

# Generate claim dataset
claim_dataset = claim_generator.generate_with_langchain_docs(
    insurance_policy_chunks[:10],  # Use more chunks for variety
    testset_size=10,
    query_distribution=claim_query_distribution
)

Applying SummaryExtractor:   0%|          | 0/9 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Applying CustomNodeFilter:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/29 [00:00<?, ?it/s]

...

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

INFO:ragas.testset.synthesizers.multi_hop.abstract:found 120 clusters
INFO:ragas.testset.synthesizers.multi_hop.specific:found 3 clusters
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.o

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

In [351]:
# Create a deep copy of the claim dataset
import copy
claim_dataset_advanced = copy.deepcopy(claim_dataset)

In [340]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
claim_dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"On June 10th, I noticed that a tree in my yard had fallen onto my roof during a storm, causing significant damage to the shingles and creating a large hole. I contacted my Shelter Insurance agent and filed a claim for the damages, hoping it would be covered under my homeowner's policy.","[HO-4 (01-07)\nFor information regarding this policy, please\ncontact your Shelter Insurance Agent.\nHOMEOWNERS'\nINSURANCE\nPOLICY\nSHELTER INSURANCE COMPANIES\nHome Office: Columbia, MO 65218-0001\ndfg\nHO-4\n(01-07)\nTO OUR CUSTOMERS – PLEASE NOTE\nPlease read this policy carefully. If you have\nquestions, contact your Shelter Agent for answers.\nNo agent can know your exact coverage needs or\nbudget considerations, so it is your responsibility to\nexamine the policy and make sure it provides the\ntypes of coverage you need in the amounts you\nrequested.\nIf you suffer a loss, please read this policy again so\nthat you will be reminded of your rights and\nobligations. It is very important for you to recognize\nthat this insurance policy is a legally binding\ncontract. If any insured fails to perform an\nobligation required by this policy, the coverage\nwhich it might otherwise provide could be lost.\n HO-4 (01-07)\nTHE INDEX\nWHERE YOU CAN FIND IT]","This claim is invalid based on the provided policy context. The policy emphasizes that it is a legally binding contract and that coverage depends on the specific terms and conditions outlined within the policy. Since the context does not specify coverage details for fallen trees or storm-related damage, and given that the policy advises contacting your Shelter Insurance agent for clarification, it is likely that such damage may be excluded or require specific coverage endorsements. To make this claim valid, you should review your policy to confirm whether fallen trees and storm damage are covered, and consult your agent for clarification on coverage for such incidents.",single_hop_specifc_query_synthesizer
1,"On June 10th, I noticed that my property was damaged due to a fire that started in my garage. I immediately contacted the insurance company to file a claim. During the claim process, I was asked to provide the declarations page, which I did. The damage was extensive, affecting my garage and the adjacent rooms. I believe this incident is covered under my policy because I have a valid declarations page listing my property details and coverage limits.","[contract. If any insured fails to perform an\nobligation required by this policy, the coverage\nwhich it might otherwise provide could be lost.\n HO-4 (01-07)\nTHE INDEX\nWHERE YOU CAN FIND IT\nDeclarations - Your Name, Location of Your Residence, Policy Period, Limits of Liability and Deductibles\nBeginning on Page\nAgreement ............................................................................................................................................................. 2\nDefinitions ............................................................................................................................................................. 2\nGeneral Agreements applicable to entire policy .................................................................................................... 8\nSection I - Property Protection ........................................................................................................................... 10]","This claim is invalid based on the provided policy context. The declarations page, which includes details such as your name, residence location, policy period, limits of liability, and deductibles, is essential for establishing coverage. However, the context indicates that coverage depends on the insured's performance of obligations outlined in the policy. Since the claim involves fire damage, coverage would depend on whether the policy explicitly covers fire-related damages and whether all obligations, such as maintaining sa

In [ ]:
def print_agent_messages(response):
    print(f"user_input: {response['user_input']}")
    for message in response['messages']:
        print(f"Message from {message.role}: {message.content}")

In [ ]:
def extract_page_content(documents, include_metadata=False, clean_text=True):
    """
    Extract page content from various document formats using robust regex parsing.
    
    Args:
        documents: String representation of Document objects or actual Document objects
        include_metadata: If True, return metadata (currently only for actual Document objects)
        clean_text: If True, clean up formatting characters
    
    Returns:
        List of page content strings
    """
    import re
    
    # If it's a string representation, use the robust regex approach
    if isinstance(documents, str):
        # Find all page_content sections using regex (from page_content_parser.py)
        pattern = r"page_content='(.*?)'\)"
        matches = re.findall(pattern, documents, re.DOTALL)
        
        if not matches:
            # Fallback: try alternative patterns for string representations
            pattern = r"page_content=['\"]([^'\"]*?)['\"]\)"
            matches = re.findall(pattern, documents, re.DOTALL)
        
        # Clean up escaped characters and formatting
        cleaned_contents = []
        for match in matches:
            if clean_text:
                # Replace escaped quotes and control characters
                cleaned = match.replace("\\'", "'").replace("\\x0c", "\n")
                cleaned = cleaned.strip()
            else:
                cleaned = match
            cleaned_contents.append(cleaned)
        
        return cleaned_contents
    
    # Handle actual Document objects (existing logic)
    elif hasattr(documents, '__iter__') and not isinstance(documents, str):
        if isinstance(documents, list):
            results = []
            for doc in documents:
                content = doc.page_content if hasattr(doc, 'page_content') else str(doc)
                if clean_text:
                    content = content.replace('\\x0c', '\n').strip()
                
                if include_metadata and hasattr(doc, 'metadata'):
                    results.append({'content': content, 'metadata': doc.metadata})
                else:
                    results.append(content)
            return results
        else:
            # Single document
            content = documents.page_content if hasattr(documents, 'page_content') else str(documents)
            if clean_text:
                content = content.replace('\\x0c', '\n').strip()
            
            if include_metadata and hasattr(documents, 'metadata'):
                return [{'content': content, 'metadata': documents.metadata}]
            else:
                return [content]
    
    else:
        raise ValueError("Input must be Document objects or string representation")

In [ ]:
# Test the updated extract_page_content function with sample data
test_string = """(metadata={'producer': 'Acrobat Distiller 7.0.5 (Windows)', 'creator': 'Adobe PageMaker 7.0'}, page_content='COVERAGE G – DAMAGE TO PROPERTY OF OTHERS\\nINSURING AGREEMENT\\nWe will pay for property damage caused by an\\ninsured to real property or personal property owned\\nby others.'), (metadata={'producer': 'Acrobat Distiller 7.0.5 (Windows)'}, page_content='HO-4 (01-07)\\nFor information regarding this policy, please\\ncontact your Shelter Insurance Agent.')"""

print("Testing updated extract_page_content function:")
print("=" * 50)

# Test string parsing
result = extract_page_content(test_string)
print(f"Extracted {len(result)} page contents:")
for i, content in enumerate(result, 1):
    print(f"\n{i}. {content[:100]}...")

print("\n" + "=" * 50)
print("Function successfully updated!")

Testing updated extract_page_content function:
Extracted 2 page contents:

1. COVERAGE G – DAMAGE TO PROPERTY OF OTHERS\nINSURING AGREEMENT\nWe will pay for property damage cause...

2. HO-4 (01-07)\nFor information regarding this policy, please\ncontact your Shelter Insurance Agent....

Function successfully updated!


In [ ]:
from langchain_core.messages import ToolMessage, AIMessage
def collect_tool_responses(response):
    """
    Collects all tool responses from a response object and returns them with their tool type.
    
    Args:
        response (dict): Response object containing messages and tool calls
        
    Returns:
        list: List of tuples containing (tool_type, tool_response)
    """
    print(f"response: {response}")
    breakpoint()
    tool_responses = []
    # Iterate through messages looking for tool messages
    for message in response.get('messages', []):
        if isinstance(message, ToolMessage):
            # Get the tool_call_id to match with the original tool call
            tool_call_id = message.tool_call_id
            # print(f"tool_call_id: {tool_call_id}")
            # print(f"message: {message}")
            # Find the corresponding AIMessage with tool call to get the tool type
            for prev_message in response.get('messages', []):
                if isinstance(prev_message, AIMessage) and prev_message.tool_calls:
                    for tool_call in prev_message.tool_calls:
                        if tool_call['id'] == tool_call_id:
                            tool_type = tool_call['name']
                            # print(message.content)
                            # print(f"type: {type(message.content)}")
                            document_content = message.content.split("Document")
                            content = ""
                            for doc in document_content:
                                print(f"content here: {content}")
                                content += extract_page_content(doc)
                                print(f"content here: {content}")
                            if "Document" in content:
                                breakpoint()
                            # print(f"content: {content}")
                            # breakpoint()
                            tool_responses.append(f"{tool_type}: {content}")
                            break

    final_messsage = response.get('messages', [])[-1]
    if isinstance(final_messsage, AIMessage):
        output = final_messsage.content
        print(f"output: {response}")
                            
    return output, tool_responses


In [ ]:
import ast
import re

def extract_page_contents_from_tool_messages(blob):
    """
    Extracts `page_content` fields from all ToolMessage content entries in the given object.
    
    Args:
        blob (dict): The original structured data containing ToolMessage objects.

    Returns:
        list of str: Extracted `page_content` strings from each document in each ToolMessage.
    """
    page_contents = []

    for message in blob.get("messages", []):
        print(f"message: {message}")
        breakpoint()
        if message.get("@type") == "ToolMessage" or message.get("tool_calls"):
            content = message.get("content", "")
            if isinstance(content, str) and "Document(" in content:
                # Extract each Document(...) block
                document_blocks = re.findall(r"Document\((.*?)\)", content, re.DOTALL)
                for block in document_blocks:
                    try:
                        # Extract page_content='...' or page_content="..."
                        match = re.search(r"page_content\s*=\s*(?P<quote>['\"])(.*?)(?P=quote)", block, re.DOTALL)
                        if match:
                            raw_content = match.group(2)
                            cleaned = raw_content.encode().decode('unicode_escape')
                            page_contents.append(cleaned.strip())
                    except Exception as e:
                        print(f"Error processing block: {e}")
                        continue

    return page_contents

In [ ]:
import ast
import re

def extract_page_contents_from_tool_messages(blob):
    """
    Extracts `page_content` fields from all ToolMessage content entries in the given object.
    
    Args:
        blob (dict): The original structured data containing ToolMessage objects.

    Returns:
        list of str: Extracted `page_content` strings from each document in each ToolMessage.
    """
    page_contents = []

    for message in blob.get("messages", []):
        if isinstance(message, ToolMessage):
            content = message.content
            print(f"content: {content}")
            breakpoint()
            if isinstance(content, str) and "Document(" in content:
                # Extract each Document(...) block
                # First convert string representation of list to actual list
                try:
                    content_list = ast.literal_eval(content)
                except Exception as e:
                    print(f"Error parsing content as list: {e}")
                    return page_contents

                # Process each Document object in the list
                for doc in content_list:
                    try:
                        # Extract page_content field
                        if hasattr(doc, 'page_content'):
                            cleaned = doc.page_content.encode().decode('unicode_escape')
                            page_contents.append(cleaned.strip())
                    except Exception as e:
                        print(f"Error processing document: {e}")
                        continue

    return page_contents

content = extract_page_contents_from_tool_messages(response)
print(f"content: {content}")

content: [Document(metadata={'producer': 'Acrobat Distiller 7.0.5 (Windows)', 'creator': 'Adobe PageMaker 7.0', 'creationdate': '2007-10-18T14:41:31-05:00', 'subject': 'HO3', 'author': 'WPD4481', 'keywords': '', 'moddate': '2007-10-18T14:41:35-04:00', 'title': 'HO3', 'source': './data/oklahoma_home_insurance_policy.pdf', 'total_pages': 29, '_id': '32aef1e14fd043ebad2a6bd3aec5a027', '_collection_name': 'insurance_policy'}, page_content='COVERAGE G – DAMAGE TO PROPERTY OF OTHERS\nINSURING AGREEMENT\nWe will pay for property damage caused by an\ninsured to real property or personal property owned\nby others.\nLIMITS OF LIABILITY\nWe will not pay more than the smallest of the\nfollowing amounts for any one occurrence:\n 1. the market value\n of the property at the time of\nloss;\n 2. the restoration cost; or\n 3. $1,000.\nEXCLUSIONS\nWe will not pay for property damage:\n1. caused by any insured who has attained the age\nof 13, unless it results from an accident;\n2. to property owned by, 

In [ ]:
def extract_context(response):
    messages = response.get("messages", [])
    context = []
    for message in messages:
        if isinstance(message, ToolMessage):
            tool_context = []
            content = message.content
            documents = content.split("Document")[0:]
        
            for document in documents:
                if "page_content" in document:
                    page_content = document.split("page_content")[1].split("'")[1]
                    tool_context.append(page_content)
            if tool_context:
                context.append(f"{message.name}: {tool_context}")
        
    return context
            
    
context = extract_context(response)
context

['retrieve_insurance_policy: [\'COVERAGE G – DAMAGE TO PROPERTY OF OTHERS\\\\nINSURING AGREEMENT\\\\nWe will pay for property damage caused by an\\\\ninsured to real property or personal property owned\\\\nby others.\\\\nLIMITS OF LIABILITY\\\\nWe will not pay more than the smallest of the\\\\nfollowing amounts for any one occurrence:\\\\n 1. the market value\\\\n of the property at the time of\\\\nloss;\\\\n 2. the restoration cost; or\\\\n 3. $1,000.\\\\nEXCLUSIONS\\\\nWe will not pay for property damage:\\\\n1. caused by any insured who has attained the age\\\\nof 13, unless it results from an accident;\\\\n2. to property owned by, or rented to:\\\\n(a) any insured;\\\\n\\\\x0cHO-4 (01-07)\\\\n(b) any tenant of an insured; or\\\\n(c) any resident of any insured’s household;\\\\n3. arising out of:\\\\n(a) any act or omission related to making\\\\npremises owned, rented, or controlled by\\\\nany insured safe, unless those premises are\\\\nthe insured premises;\\\\n(b) the business of 

### Evaluation

In [341]:
claim_dataset

Testset(samples=[TestsetSample(eval_sample=SingleTurnSample(user_input="On June 10th, I noticed that a tree in my yard had fallen onto my roof during a storm, causing significant damage to the shingles and creating a large hole. I contacted my Shelter Insurance agent and filed a claim for the damages, hoping it would be covered under my homeowner's policy.", retrieved_contexts=None, reference_contexts=["HO-4 (01-07)\nFor information regarding this policy, please\ncontact your Shelter Insurance Agent.\nHOMEOWNERS'\nINSURANCE\nPOLICY\nSHELTER INSURANCE COMPANIES\nHome Office: Columbia, MO 65218-0001\ndfg\nHO-4\n(01-07)\nTO OUR CUSTOMERS – PLEASE NOTE\nPlease read this policy carefully.  If you have\nquestions, contact your Shelter Agent for answers.\nNo agent can know your exact coverage needs or\nbudget considerations, so it is your responsibility to\nexamine the policy and make sure it provides the\ntypes of coverage you need in the amounts you\nrequested.\nIf you suffer a loss, please

In [342]:
from ragas import EvaluationDataset
EvaluationDataset.from_list(claim_dataset.to_list()).to_pandas()


,user_input,reference_contexts,reference
0,"On June 10th, I noticed that a tree in my yard had fallen onto my roof during a storm, causing significant damage to the shingles and creating a large hole. I contacted my Shelter Insurance agent and filed a claim for the damages, hoping it would be covered under my homeowner's policy.","[HO-4 (01-07)\nFor information regarding this policy, please\ncontact your Shelter Insurance Agent.\nHOMEOWNERS'\nINSURANCE\nPOLICY\nSHELTER INSURANCE COMPANIES\nHome Office: Columbia, MO 65218-0001\ndfg\nHO-4\n(01-07)\nTO OUR CUSTOMERS – PLEASE NOTE\nPlease read this policy carefully. If you have\nquestions, contact your Shelter Agent for answers.\nNo agent can know your exact coverage needs or\nbudget considerations, so it is your responsibility to\nexamine the policy and make sure it provides the\ntypes of coverage you need in the amounts you\nrequested.\nIf you suffer a loss, please read this policy again so\nthat you will be reminded of your rights and\nobligations. It is very important for you to recognize\nthat this insurance policy is a legally binding\ncontract. If any insured fails to perform an\nobligation required by this policy, the coverage\nwhich it might otherwise provide could be lost.\n HO-4 (01-07)\nTHE INDEX\nWHERE YOU CAN FIND IT]","This claim is invalid based on the provided policy context. The policy emphasizes that it is a legally binding contract and that coverage depends on the specific terms and conditions outlined within the policy. Since the context does not specify coverage details for fallen trees or storm-related damage, and given that the policy advises contacting your Shelter Insurance agent for clarification, it is likely that such damage may be excluded or require specific coverage endorsements. To make this claim valid, you should review your policy to confirm whether fallen trees and storm damage are covered, and consult your agent for clarification on coverage for such incidents."
1,"On June 10th, I noticed that my property was damaged due to a fire that started in my garage. I immediately contacted the insurance company to file a claim. During the claim process, I was asked to provide the declarations page, which I did. The damage was extensive, affecting my garage and the adjacent rooms. I believe this incident is covered under my policy because I have a valid declarations page listing my property details and coverage limits.","[contract. If any insured fails to perform an\nobligation required by this policy, the coverage\nwhich it might otherwise provide could be lost.\n HO-4 (01-07)\nTHE INDEX\nWHERE YOU CAN FIND IT\nDeclarations - Your Name, Location of Your Residence, Policy Period, Limits of Liability and Deductibles\nBeginning on Page\nAgreement ............................................................................................................................................................. 2\nDefinitions ............................................................................................................................................................. 2\nGeneral Agreements applicable to entire policy .................................................................................................... 8\nSection I - Property Protection ........................................................................................................................... 10]","This claim is invalid based on the provided policy context. The declarations page, which includes details such as your name, residence location, policy period, limits of liability, and deductibles, is essential for establishing coverage. However, the context indicates that coverage depends on the insured's performance of obligations outlined in the policy. Since the claim involves fire damage, coverage would depend on whether the policy explicitly covers fire-related damages and whether all obligations, such as maintaining safety measures, were fulfilled. Without specific policy

#### Initial Prototype

In [352]:
import copy
evaluated_dataset = copy.deepcopy(claim_dataset)
for test_row in list(evaluated_dataset):
    result = claim_consultant_agent.invoke({"user_input" : test_row.eval_sample.user_input})
    context = extract_context(result)
    test_row.eval_sample.retrieved_contexts = context
    test_row.eval_sample.response = result["messages"][-1].content


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request

In [320]:
EvaluationDataset.from_list(evaluated_dataset).to_pandas()

TypeError: ragas.dataset_schema.SingleTurnSample() argument after ** must be a mapping, not TestsetSample

In [353]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(evaluated_dataset.to_pandas())

In [325]:
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

In [326]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig
from ragas import evaluate

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:

{'context_recall': 0.5118, 'faithfulness': 0.4081, 'factual_correctness(mode=f1)': 0.4645, 'answer_relevancy': 0.4877, 'context_entity_recall': 0.1000, 'noise_sensitivity(mode=relevant)': 0.1312}

In [358]:
eval_llm = ChatOpenAI(model="gpt-4.1")

In [359]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

In [361]:
evaluate(
    claim_consultant_agent.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "basic_claim_consultant_agent"},
)

View the evaluation results for experiment: 'essential-history-13' at:
https://smith.langchain.com/o/c2cfcbd8-d5df-509f-8f0e-973ec8ab5a6b/datasets/f26a53d0-a557-462a-9f9e-6666034c736f/compare?selectedSessions=cd8f20dc-6f33-4d55-a052-7d07988c4e44




0it [00:00, ?it/s]

ERROR:langsmith.evaluation._runner:Error running target function: 'user_input'
Traceback (most recent call last):
  File "/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langsmith/evaluation/_runner.py", line 1924, in _forward
    fn(*args, langsmith_extra=langsmith_extra)
  File "/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/main.py", line 3019, in invoke
    stream_mode=["updates", "values"]
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/main.py", line 2651, in stream
    )
      
  File "/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/_runner.py", line 162, in tick
    run_with_retry(
  File "/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/_retry.py", line 42, in run_with_retry
    return task.proc.invoke(

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,"I recently had a guest at my home who slipped and fell on my driveway, injuring their ankle. They required medical treatment, and I want to know if my homeowner's insurance covers their medical expenses and any potential liability I might have. Additionally, I noticed some damage to my neighbor's fence caused by a fallen tree from my property during a storm. Can I be held liable for this damage, and is it covered under my policy? I also want to understand if my coverage includes protection against claims for property damage and personal injuries caused by my property or actions.",None,"KeyError('user_input')\n\nTraceback (most recent call last):\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/main.py"", line 3019, in invoke\n stream_mode=[""updates"", ""values""]\n ^^^^^^^^^^^^^^^^^^^^^^^^\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/main.py"", line 2651, in stream\n )\n \n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/_runner.py"", line 162, in tick\n run_with_retry(\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/_retry.py"", line 42, in run_with_retry\n return task.proc.invoke(task.input, config)\n ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/_internal/_runnable.py"", line 646, in invoke\n # others are writers which don't need to be run in context\n ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/_internal/_runnable.py"", line 390, in invoke\n else:\n ^^^^\n File ""/var/folders/r5/wsbp5_jj7lsfh6rxrw7p7lv00000gn/T/ipykernel_16029/3022482314.py"", line 40, in prepare_input\n messages.append(HumanMessage(content=state[""user_input""]))\n ~~~~~^^^^^^^^^^^^^^\nKeyError: 'user_input'\nDuring task with name 'prepare_input' and id 'ef6b2fc6-faab-5e9a-4062-a5ea8e5e6e78'\n","This claim involves multiple coverage aspects of the HO-4 policy, specifically personal liability, medical payments, and property damage. The guest's injury on the driveway falls under Coverage F - Medical Payments to Others, which generally covers medical expenses regardless of fault, so the medical treatment would likely be covered. Regarding liability for the neighbor's fence damage caused by a fallen tree, Coverage G - Damage to Property of Others, would typically cover such damage if the fallen tree was from your property and caused damage during a storm. However, the policy's general provisions in Section II specify that coverage applies if the damage results from an insured peril, such as a storm. If the storm was an insured peril, then liability and property damage claims would be covered. Based on the provided policy sections, the claim appears valid, assuming the storm is an insured peril and the injury was accidental. To strengthen the claim, ensure that the damage was caused by a covered peril and that the injury was reported promptly. Therefore, the claim is valid under the policy's personal liability, medical payments, and property damage coverages, provided the conditions are met.",0,0,0,0.005486,0e56a201-b5e3-4b45-b994-b398dcd865c8,3669d8b1-0393-4335-9c7a-39e07cffe587
1,"I got hurt at my house when my neighbor's kid was playing and he fell and broke his arm, and I think I should get money for that because I have coverage C for my stuff and coverage F for medical stuff, but I didn't call the doctor or anything, and I think I should be covered because it happened at my house and I didn't do anything wrong, so I want to claim for the medical bills and maybe for the liability too, even tho

#### Advanced Prototype

In [354]:
advanced_evaluated_dataset = copy.deepcopy(claim_dataset)
for test_row in list(advanced_evaluated_dataset):
    result = advanced_claim_consultant_agent.invoke({"user_input" : test_row.eval_sample.user_input})
    context = extract_context(result)
    test_row.eval_sample.retrieved_contexts = context
    test_row.eval_sample.response = result["messages"][-1].content


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST

In [355]:
advanced_evaluated_dataset.to_pandas()

user_input  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           On June 10th, I noticed that my roof had a large hole after a severe storm, which caused water to leak into my living room and damage the ceiling and furniture. The storm was very intense, and I believe this damage is covered under my Shelter Insurance HO-4 policy. I filed a claim immediately and requested repairs.   
1                                                                                                                                                                                                                                                                                                                                                                                                  On April 10th, I noticed that my small retail shop's window was broken due to a storm, and I immediately filed a claim. I believe this damage should be covered under Section I of my policy, as it pertains to property protection. The storm caused the window to shatter, and I need to replace it to continue my business operations. I want to know if this damage is covered and how to proceed with the claim.   
2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              I was at my friend's house last weekend when I accidentally knocked over a garden statue that was on their property. The statue broke into pieces, and my friend asked me to pay for the damages. I want to file a claim under my homeowner's insurance for the damage to their property.   
3                                                                                                                                                                                                         I am a homeowner who occasionally sells homemade crafts from my residence and also runs a small consulting business from my home office. Recently, I had a minor accident where I accidentally caused property damage to a neighbor's fence while delivering a large order of crafts. Additionally, I received a request from a client for consulting services that I provided during work hours at my home. I want to understand if my insurance policy covers the damage caused during my craft deliveries and the consulting activities, considering the definitions of accident and business in my policy.   
4                                                                                                                            A homeowner reports that during a recent storm, a tree on their property was uprooted and fell onto their house, causing significant roof damage. The homeowner claims that the tree fell suddenly during the storm, and the damage occurred immediately. They also mention that the tree was old and had some signs of disease, but they had not taken any action to remove it. The homeowner is unsure if the damage is covered, as they had previously noticed some minor cracks in the roof but did not report or repair them. They want to know if the policy covers the damage caused by the fallen tree, considering the age and condition of the tree and the timing of the damage.   
5     

In [356]:
advanced_evaluation_dataset = EvaluationDataset.from_pandas(advanced_evaluated_dataset.to_pandas())

In [357]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig
from ragas import evaluate

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=advanced_evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:

{'context_recall': 0.6281, 'faithfulness': 0.2334, 'factual_correctness(mode=f1)': 0.4264, 'answer_relevancy': 0.7339, 'context_entity_recall': 0.0972, 'noise_sensitivity(mode=relevant)': 0.0685}